# ATS Classifier, NLP Case

This is a prototype. The accuracy might be not great, Bear with it.

check for GPU (Only when using GPU)

In [ ]:
!nvidia-smi

Sun Dec  8 10:57:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# !pip install spacy

## Import Libraries

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from transformers.models.auto import AutoTokenizer, AutoModel
from transformers import pipeline
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy as np
from typing import List, Dict
import json
# from spacy

set device (CPU or CUDA)

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 10

## Only for Colab, load google drive

In [ ]:
USE_COLAB = True    # change to 'False' if not using Colab
from google.colab import drive

if USE_COLAB:
  drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import time
def timer_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Execution time: {execution_time:.4f} seconds")
        return result
    return wrapper

## Load Dataset

### Option 1: Load TXT Dataset of CV (File Tree Structure)

In [ ]:
# import os

# file_path = "ngko isi dhewe"

# train = []
# val = []

# for split in ['train', 'val']:
#     split_path = os.path.join(file_path, split)
#     if not os.path.exists(split_path):
#         continue  # Skip if the directory doesn't exist

#     # Iterate through 'accepted' and 'declined' directories
#     for label_dir in os.listdir(split_path):
#         label_path = os.path.join(split_path, label_dir)
#         if not os.path.isdir(label_path):
#             continue  # Skip if not a directory

#         # Assign label based on folder name
#         label = 1 if label_dir == 'accepted' else 0

#         # Iterate through text files in the label directory
#         for txt_file in os.listdir(label_path):
#             if not txt_file.endswith('.txt'):
#                 continue  # Skip non-txt files

#             # Read the file content
#             file_full_path = os.path.join(label_path, txt_file)
#             with open(file_full_path, 'r') as file:
#                 text = file.read()

#             # Append to the corresponding list
#             if split == 'train':
#                 train.append({'file': file_full_path, 'text': text, 'label': label})
#             else:
#                 val.append({'file': file_full_path, 'text': text, 'label': label})



### Option 2: CSV file structure (CV text is separated from the CSV)

In [ ]:
# import gc
# import os

# df = pd.read_csv('{file_path}/dataset.csv'.format(file_path=file_path))

# dataset = {'cv_text':[], 'applied_job': [], 'qualified': [] }

# for cv_file in df['cv_file']:
#     with open(cv_file, 'r') as file:
#         text = file.read()
#         dataset['text'].append(text)
#         dataset['applied_job'].append(df[df['cv_file'] == cv_file]['applied_job'].values[0])
#         dataset['qualified'].append(df[df['cv_file'] == cv_file]['qualified'].values[0])

# df = pd.DataFrame(dataset)
# del dataset
# gc.collect()

# train, val = train_test_split(df, test_size=0.2, random_state=42)

### Option 3: CSV File structure (CV text is within the CSV file)

In [ ]:
dataset = pd.read_csv('{file_path}/dataset.csv'.format(file_path=file_path))

train, val = train_test_split(dataset, test_size=0.2, random_state=42)

## Create PyTorch Dataset

In [ ]:
class CVDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return {
            'text': item['text'],
            'applied_job': torch.tensor(item['label'], dtype=torch.long)
        }

## Model Class

In [ ]:
class ATSJobClassifier(nn.Module):
    def __init__(self, transformer_model, n_class, job_embed_dim=16):
        super(ATSJobClassifier, self).__init__()
        self.transformer_model = transformer_model

        transformer_hidden_size = transformer_model.config.hidden_size

        self.job_embedding = nn.EmbeddingBag(job_embed_dim, transformer_hidden_size, mode='mean', sparse=True)

        self.classifier = nn.Linear(transformer_hidden_size, n_class)

    def forward(self, input_ids, attention_mask, applied_job=None):
        transformer_output = self.transformer_model(input_ids=input_ids, attention_mask=attention_mask)
        transformer_hidden_state = transformer_output.last_hidden_state[:, 0, :]

        job_embedded = self.job_embedding(applied_job)

        if applied_job is not None:
            job_embedded = self.job_embedding(applied_job)
            x = transformer_hidden_state + job_embedded
        else:
            x = transformer_hidden_state

        logits = self.classifier(x)
        return torch.softmax(logits, dim=1)

In [ ]:
def clean_punctuation_except_period_and_colon(text):
  cleaned_text = re.sub(r'[^a-zA-Z0-9\s\.\,\:]', '', text)
  return cleaned_text

class IndonesianCVMatcher():
    def __init__(self,  job_req_dict, model_name='indolem/indobert-base-uncased', ner_model_name='cahya/bert-base-indonesian-NER', stop_words = None):
      super(IndonesianCVMatcher, self).__init__()
      self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.model = AutoModel.from_pretrained(model_name).to(self.device)
      self.job_req_dict = job_req_dict
      self.stop_words = stop_words
      self.ner_pipeline = pipeline('ner', model=ner_model_name, tokenizer=ner_model_name, grouped_entities=True)

    def preprocess_text(self, text) -> str:
      # Convert text to lowercase
      text = text.lower()
      # Remove punctuation but not number
      text = self.clean_punctuation_except_period_and_colon(text)

      text = re.sub(r'\s+', ' ', text)


      # Remove extra whitespace
      text = ' '.join(text.split())

      # lemmatization


      # Remove stopwords
      if self.stop_words is not None:
        words = text.split()
        words = [word for word in words if word not in self.stop_words]

        return ' '.join(words)

      return text

    def get_bert_embeddings(self, text: str) -> np.ndarray:
      # Tokenize and prepare input
      inputs = self.tokenizer(
          text,
          max_length=512,
          padding=True,
          truncation=True,
          return_tensors='pt'
      ).to(self.device)

      # Get embeddings
      with torch.no_grad():
          outputs = self.model(**inputs)
          embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()

      return embeddings[0]

    def extract_skills(self, text: str) -> List[str]:
      # Common Indonesian technical terms and patterns
      # text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
      text = clean_punctuation_except_period_and_colon(text)
      text = re.sub(r'\s+', ' ', text)
      text = ' '.join(text.split())
      text = text.lower()

      skill_patterns = [
        r'(?i)(?:ahli|mampu|menguasai|berpengalaman dalam)\s+([\w\s]+)',
        r'(?i)(?:keahlian|kemampuan|keterampilan)[\s:]+([^.]+)',
        r'(?i)(?:sertifikasi|sertifikat)\s+([\w\s]+)',
        r'(?i)pengalaman\s+(?:dengan|dalam)\s+([\w\s]+)',
      ]

      skills = []
      for pattern in skill_patterns:
        matches = re.finditer(pattern, text)
        for match in matches:
          skill = match.group(1).strip().lower()
          if len(skill) > 0:
            skills.append(skill)
      # print(f"Skills: {skills}")
      return list(set(skills))

    def extract_named_entities(self, text: str) -> Dict[str, List[str]]:
        text = clean_punctuation_except_period_and_colon(text)
        inputs = self.tokenizer(text, truncation=True,padding=True, max_length=512)
        decoded = self.tokenizer.decode(inputs['input_ids'])

        # Updated line to pass the decoded text using the 'text' key:
        entities = self.ner_pipeline(decoded)
        entity_dict = {}
        for entity in entities:
            label = entity['entity_group']
            entity_text = entity['word'].strip()
            if label not in entity_dict:
                entity_dict[label] = []
            entity_dict[label].append(entity_text)
        # Ensure no duplicate entities
        for key in entity_dict:
            entity_dict[key] = list(set(entity_dict[key]))
        # print("Entities:", entity_dict)
        return entity_dict

    def calculate_entity_match_score(self, cv_entities, job_entities):
        entity_match_score = 0
        matching_details = {}

        # Find matching labels (keys in both dictionaries)
        matching_labels = set(cv_entities.keys()).intersection(job_entities.keys())

        for label in matching_labels:
            # Find common and total entities for the current label
            common_entities = set(cv_entities[label]).intersection(job_entities[label])
            total_entities = set(cv_entities[label]).union(job_entities[label])

            if total_entities:  # Avoid division by zero
                match_score = len(common_entities) / len(total_entities)
                entity_match_score += match_score
                matching_details[label] = list(common_entities)  # Store matched entities for each label

        # Average the score across all matching labels
        if matching_labels:
            entity_match_score /= len(matching_labels)

        return entity_match_score, matching_details


    def extract_ipk(self, text: str) -> float:
        text = re.sub(r'[^a-zA-Z0-9\s\.]', ' ', text)
        angka = r'([0-4](?:\.\d{1,2})?)(?:/\s*[0-4](?:\.\d{1,3})?)?'
        # matches = re.findall(angka, text.lower())
        # for match in matches:
        #     try:
        #         # print(match[0])
        #         ipk = float(match[0])
        #     except ValueError:
        #         pass
        # pattern = r'(?:ipk(?:\s*[:\s]*minimal|\s*:)?\s*|)'+angka
        pattern = r'(?i)\bipk\b.*?([0-4](?:\.\d{1,3})?)'
        matches = re.findall(pattern, text)
        if matches:
            # print(matches)
            return max(float(match) for match in matches)
        return 0.0

    @timer_decorator
    def calculate_similarity_scores(self, cv_text: str, job_req: str, mode:str = 'embedding') -> Dict[str, float]:
      cv_processed = self.preprocess_text(cv_text)
      job_processed = self.preprocess_text(job_req)

      # cv_text = clean_punctuation_except_period_and_colon(cv_text)
      # job_req = clean_punctuation_except_period_and_colon(job_req)

      cv_embedding = self.get_bert_embeddings(cv_text)
      job_embedding = self.get_bert_embeddings(job_req)

      similarity_score = cosine_similarity([cv_embedding], [job_embedding])[0][0]

      cv_skills = self.extract_skills(cv_text)
      job_skills = self.extract_skills(job_req)

       # Extract named entities
      cv_entities = self.extract_named_entities(cv_text)
      job_entities = self.extract_named_entities(job_req)

      # Calculate entity match score (e.g., job titles, organizations)
      entity_match_score, _ = self.calculate_entity_match_score(cv_entities, job_entities)

      if cv_skills and job_skills:
          # Combine skills into a single string for embedding calculation
          if mode == 'embedding':
            cv_skills_text = ' '.join(cv_skills)
            job_skills_text = ' '.join(job_skills)

            cv_skills_embedding = self.get_bert_embeddings(cv_skills_text)
            job_skills_embedding = self.get_bert_embeddings(job_skills_text)
            skills_similarity = cosine_similarity([cv_skills_embedding], [job_skills_embedding])[0][0]
            skills_similarity_factor = 1000
            keyword_overlap_factor = 100
          elif mode == 'tf-idf':
            vectorizer = TfidfVectorizer()
            skill_corpus = [' '.join(cv_skills), ' '.join(job_skills)]
            tfidf_matrix = vectorizer.fit_transform(skill_corpus)
            skills_similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0]
            skills_similarity_factor = 10
            keyword_overlap_factor = 1
          else:
            raise ValueError("Invalid mode. Choose 'embedding' or 'tf-idf'.")
      else:
          skills_similarity = 0

      # Calculate keyword overlap
      cv_words = set(cv_processed.split())
      job_words = set(job_processed.split())
      keyword_overlap = len(cv_words.intersection(job_words)) / len(job_words)

      # Calculate the overall score
      # overall_score = (similarity_score + keyword_overlap + skills_similarity**3 + entity_match_score**(0.1)+3) / (4+3)
      overall_score = (similarity_score*0.04 + (skills_similarity)*0.75 + keyword_overlap*0.2 + entity_match_score*0.05 + compensation_value)/(1+compensation_value)

      return {
            'semantic_similarity': float(similarity_score),
            'skills_similarity': float(skills_similarity),
            'keyword_overlap': float(keyword_overlap),
            'entity_match_score': float(entity_match_score),
            'overall_score': float(overall_score)

      }

    def extract_years_of_experience(self, text: str) -> int:
        pattern = r'(\d+)\s+(tahun|years)'
        matches = re.findall(pattern, text.lower())
        if matches:
            return max(int(match[0]) for match in matches)
        return 0

    def print_match_requirements(self, cv_text: str, job_req: str, file_handle=None):
        cv_processed = self.preprocess_text(cv_text)
        job_processed = self.preprocess_text(job_req)

        # Extract years of experience
        cv_experience = self.extract_years_of_experience(cv_text)
        job_experience = self.extract_years_of_experience(job_req)

        # Extract skills
        cv_skills = self.extract_skills(cv_text)
        job_skills = self.extract_skills(job_req)

        # Extract named entities
        cv_entities = self.extract_named_entities(cv_text)
        job_entities = self.extract_named_entities(job_req)

        # Calculate entity match score and get matching details
        entity_match_score, matching_details = self.calculate_entity_match_score(cv_entities, job_entities)

        cv_ipk = self.extract_ipk(cv_text)
        job_ipk = self.extract_ipk(job_req)

        # Find overlapping skills
        matched_skills = set(cv_skills).intersection(set(job_skills))

        output = []

        output.append("\nMatched Requirements:")
        if cv_experience >= job_experience:
            output.append(f"- Required experience met: {cv_experience} years (required: {job_experience} years)")
        else:
            output.append(f"- Experience mismatch: {cv_experience} years (required: {job_experience} years)")

        if matched_skills:
            output.append(f"- Overlapping skills: {', '.join(matched_skills)}")
        else:
            output.append("- No overlapping skills found.")

        if job_ipk > 0:
            if cv_ipk >= job_ipk:
                output.append(f"- Required IPK met: {cv_ipk} (required: {job_ipk})")
            else:
                output.append(f"- IPK mismatch: {cv_ipk} (required: {job_ipk})")
        else:
            output.append("- No specific IPK requirement found in the job description.")

        output.append("\nEntity Matching Details:")
        if matching_details:
            for label, entities in matching_details.items():
                if entities:
                    output.append(f"- {label}: {', '.join(entities)}")
        else:
            output.append("- No matching entities found.")

        output.append("\nJob Requirement Skills:")
        output.append(f"- {', '.join(job_skills)}")

        output.append("\nCV Skills:")
        output.append(f"- {', '.join(cv_skills)}")

        result = "\n".join(output)

        if file_handle:
            file_handle.write(result + "\n")
        else:
            print(result)

    def rank_candidates(self, cvs: List[Dict], job_req: str) -> List[Dict]:
        rankings = []

        for cv in cvs:
            scores = self.calculate_similarity_scores(cv['text'], job_req)
            rankings.append({
                'candidate_id': cv.get('id', ''),
                'candidate_name': cv.get('name', ''),
                'scores': scores
            })

        # Sort by overall score
        rankings.sort(key=lambda x: x['scores']['overall_score'], reverse=True)
        return rankings



In [ ]:
import pickle


class IndonesianCVMatcherWithNormalizer():
    def __init__(self,  job_req_dict, model_name='indolem/indobert-base-uncased', ner_model_name='cahya/bert-base-indonesian-NER', stop_words = None, normalization_weights_path=None):
        super(IndonesianCVMatcherWithNormalizer, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(self.device)
        self.job_req_dict = job_req_dict
        self.stop_words = stop_words
        self.ner_pipeline = pipeline('ner', model=ner_model_name, tokenizer=ner_model_name, grouped_entities=True)

        if normalization_weights_path:
            with open(normalization_weights_path, 'rb') as f:
                data = pickle.load(f)
                self.min_score = data['min_score']
                self.max_score = data['max_score']
        else:
            self.min_score = float('inf')  # Start with a very high value
            self.max_score = float('-inf')

    def normalize_score(self, raw_score):
        """
        Normalize the score to a 0-1 range based on the observed min and max scores.
        """
        if self.max_score > self.min_score:
            return (raw_score - self.min_score) / (self.max_score - self.min_score)
        else:
            return 0.5  # Default to a midpoint value if no variation yet

    def update_score_bounds(self, raw_score):
        """
        Update the observed min and max scores.
        """
        self.min_score = min(self.min_score, raw_score)
        self.max_score = max(self.max_score, raw_score)

    def clean_punctuation_except_period_and_colon(self, text):
        cleaned_text = re.sub(r'[^a-zA-Z0-9\s\.\,\:]', '', text)
        return cleaned_text

    def export_normalization_weights(self, filepath):
        """
        Export normalization weights, min_score, and max_score to a file.
        """
        data = {
            'min_score': self.min_score,
            'max_score': self.max_score
        }
        with open(filepath, 'wb') as f:
            pickle.dump(data, f)


    def preprocess_text(self, text) -> str:
      # Convert text to lowercase
      text = text.lower()
      # Remove punctuation but not number
      text = self.clean_punctuation_except_period_and_colon(text)

      text = re.sub(r'\s+', ' ', text)


      # Remove extra whitespace
      text = ' '.join(text.split())

      # lemmatization


      # Remove stopwords
      if self.stop_words is not None:
        words = text.split()
        words = [word for word in words if word not in self.stop_words]

        return ' '.join(words)

      return text

    def get_bert_embeddings(self, text: str) -> np.ndarray:
      # Tokenize and prepare input
      inputs = self.tokenizer(
          text,
          max_length=512,
          padding=True,
          truncation=True,
          return_tensors='pt'
      ).to(self.device)

      # Get embeddings
      with torch.no_grad():
          outputs = self.model(**inputs)
          embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()

      return embeddings[0]

    def extract_skills(self, text: str) -> List[str]:
      # Common Indonesian technical terms and patterns
      # text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
      text = self.clean_punctuation_except_period_and_colon(text)
      text = re.sub(r'\s+', ' ', text)
      text = ' '.join(text.split())
      text = text.lower()

      skill_patterns = [
        r'(?i)(?:ahli|mampu|menguasai|berpengalaman dalam)\s+([\w\s]+)',
        r'(?i)(?:keahlian|kemampuan|keterampilan)[\s:]+([^.]+)',
        r'(?i)(?:sertifikasi|sertifikat)\s+([\w\s]+)',
        r'(?i)pengalaman\s+(?:dengan|dalam)\s+([\w\s]+)',
      ]

      skills = []
      for pattern in skill_patterns:
        matches = re.finditer(pattern, text)
        for match in matches:
          skill = match.group(1).strip().lower()
          if len(skill) > 0:
            skills.append(skill)
      # print(f"Skills: {skills}")
      return list(set(skills))

    def extract_named_entities(self, text: str) -> Dict[str, List[str]]:
        text = self.clean_punctuation_except_period_and_colon(text)
        inputs = self.tokenizer(text, truncation=True,padding=True, max_length=512)
        decoded = self.tokenizer.decode(inputs['input_ids'])

        # Updated line to pass the decoded text using the 'text' key:
        entities = self.ner_pipeline(decoded)
        entity_dict = {}
        for entity in entities:
            label = entity['entity_group']
            entity_text = entity['word'].strip()
            if label not in entity_dict:
                entity_dict[label] = []
            entity_dict[label].append(entity_text)
        # Ensure no duplicate entities
        for key in entity_dict:
            entity_dict[key] = list(set(entity_dict[key]))
        # print("Entities:", entity_dict)
        return entity_dict

    def calculate_entity_match_score(self, cv_entities, job_entities):
        entity_match_score = 0
        matching_details = {}

        # Find matching labels (keys in both dictionaries)
        matching_labels = set(cv_entities.keys()).intersection(job_entities.keys())

        for label in matching_labels:
            # Find common and total entities for the current label
            common_entities = set(cv_entities[label]).intersection(job_entities[label])
            total_entities = set(cv_entities[label]).union(job_entities[label])

            if total_entities:  # Avoid division by zero
                match_score = len(common_entities) / len(total_entities)
                entity_match_score += match_score
                matching_details[label] = list(common_entities)  # Store matched entities for each label

        # Average the score across all matching labels
        if matching_labels:
            entity_match_score /= len(matching_labels)

        return entity_match_score, matching_details


    def extract_ipk(self, text: str) -> float:
        text = re.sub(r'[^a-zA-Z0-9\s\.]', ' ', text)
        angka = r'([0-4](?:\.\d{1,2})?)(?:/\s*[0-4](?:\.\d{1,3})?)?'
        # matches = re.findall(angka, text.lower())
        # for match in matches:
        #     try:
        #         # print(match[0])
        #         ipk = float(match[0])
        #     except ValueError:
        #         pass
        # pattern = r'(?:ipk(?:\s*[:\s]*minimal|\s*:)?\s*|)'+angka
        pattern = r'(?i)\bipk\b.*?([0-4](?:\.\d{1,3})?)'
        matches = re.findall(pattern, text)
        if matches:
            # print(matches)
            return max(float(match) for match in matches)
        return 0.0

    def export(self, filepath):
        """
        Export the entire object, including model, tokenizer, normalization weights, and score bounds.
        """
        data = {
            'model': self.model,
            'tokenizer': self.tokenizer,
            'min_score': self.min_score,
            'max_score': self.max_score
        }
        with open(filepath, 'wb') as f:
            pickle.dump(data, f)

    @classmethod
    def _import(cls, filepath):
        """
        Import the entire object, including model, tokenizer, normalization weights, and score bounds.
        """
        with open(filepath, 'rb') as f:
            data = pickle.load(f)
            obj = cls(job_req_dict=None)  # Create an instance without passing the job_req_dict
            obj.model = data['model']
            obj.tokenizer = data['tokenizer']
            obj.min_score = data['min_score']
            obj.max_score = data['max_score']
            return obj


    def calculate_similarity_scores(self, cv_text: str, job_req: str, mode: str = 'embedding') -> Dict[str, float]:
        cv_processed = self.preprocess_text(cv_text)
        job_processed = self.preprocess_text(job_req)

        cv_embedding = self.get_bert_embeddings(cv_text)
        job_embedding = self.get_bert_embeddings(job_req)

        similarity_score = cosine_similarity([cv_embedding], [job_embedding])[0][0]

        cv_skills = self.extract_skills(cv_text)
        job_skills = self.extract_skills(job_req)

        cv_entities = self.extract_named_entities(cv_text)
        job_entities = self.extract_named_entities(job_req)

        entity_match_score, _ = self.calculate_entity_match_score(cv_entities, job_entities)

        if cv_skills and job_skills:
            if mode == 'embedding':
                cv_skills_text = ' '.join(cv_skills)
                job_skills_text = ' '.join(job_skills)

                cv_skills_embedding = self.get_bert_embeddings(cv_skills_text)
                job_skills_embedding = self.get_bert_embeddings(job_skills_text)
                skills_similarity = cosine_similarity([cv_skills_embedding], [job_skills_embedding])[0][0]
            elif mode == 'tf-idf':
                vectorizer = TfidfVectorizer()
                skill_corpus = [' '.join(cv_skills), ' '.join(job_skills)]
                tfidf_matrix = vectorizer.fit_transform(skill_corpus)
                skills_similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0]
            else:
                raise ValueError("Invalid mode. Choose 'embedding' or 'tf-idf'.")
        else:
            skills_similarity = 0

        cv_words = set(cv_processed.split())
        job_words = set(job_processed.split())
        keyword_overlap = len(cv_words.intersection(job_words)) / len(job_words)

        compensation_value = 0
        raw_overall_score = (
            similarity_score * 0.05 +
            (skills_similarity) * 0.7 +
            keyword_overlap * 0.2 +
            entity_match_score * 0.05 +
            compensation_value
        ) / (1 + compensation_value)

        # Update score bounds and normalize
        self.update_score_bounds(raw_overall_score)
        normalized_overall_score = self.normalize_score(raw_overall_score)

        return {
            'semantic_similarity': float(similarity_score),
            'skills_similarity': float(skills_similarity),
            'keyword_overlap': float(keyword_overlap),
            'entity_match_score': float(entity_match_score),
            'raw_overall_score': float(raw_overall_score),
            'normalized_overall_score': float(normalized_overall_score)
        }

    def extract_years_of_experience(self, text: str) -> int:
        pattern = r'(\d+)\s+(tahun|years)'
        matches = re.findall(pattern, text.lower())
        if matches:
            return max(int(match[0]) for match in matches)
        return 0

    def print_match_requirements(self, cv_text: str, job_req: str, file_handle=None):
        cv_processed = self.preprocess_text(cv_text)
        job_processed = self.preprocess_text(job_req)

        # Extract years of experience
        cv_experience = self.extract_years_of_experience(cv_text)
        job_experience = self.extract_years_of_experience(job_req)

        # Extract skills
        cv_skills = self.extract_skills(cv_text)
        job_skills = self.extract_skills(job_req)

        # Extract named entities
        cv_entities = self.extract_named_entities(cv_text)
        job_entities = self.extract_named_entities(job_req)

        # Calculate entity match score and get matching details
        entity_match_score, matching_details = self.calculate_entity_match_score(cv_entities, job_entities)

        cv_ipk = self.extract_ipk(cv_text)
        job_ipk = self.extract_ipk(job_req)

        # Find overlapping skills
        matched_skills = set(cv_skills).intersection(set(job_skills))

        output = []

        output.append("\nMatched Requirements:")
        if cv_experience >= job_experience:
            output.append(f"- Required experience met: {cv_experience} years (required: {job_experience} years)")
        else:
            output.append(f"- Experience mismatch: {cv_experience} years (required: {job_experience} years)")

        if matched_skills:
            output.append(f"- Overlapping skills: {', '.join(matched_skills)}")
        else:
            output.append("- No overlapping skills found.")

        if job_ipk > 0:
            if cv_ipk >= job_ipk:
                output.append(f"- Required IPK met: {cv_ipk} (required: {job_ipk})")
            else:
                output.append(f"- IPK mismatch: {cv_ipk} (required: {job_ipk})")
        else:
            output.append("- No specific IPK requirement found in the job description.")

        output.append("\nEntity Matching Details:")
        if matching_details:
            for label, entities in matching_details.items():
                if entities:
                    output.append(f"- {label}: {', '.join(entities)}")
        else:
            output.append("- No matching entities found.")

        output.append("\nJob Requirement Skills:")
        output.append(f"- {', '.join(job_skills)}")

        output.append("\nCV Skills:")
        output.append(f"- {', '.join(cv_skills)}")

        result = "\n".join(output)

        if file_handle:
            file_handle.write(result + "\n")
        else:
            print(result)

    def rank_candidates(self, cvs: List[Dict], job_req: str) -> List[Dict]:
        rankings = []

        for cv in cvs:
            scores = self.calculate_similarity_scores(cv['text'], job_req)
            rankings.append({
                'candidate_id': cv.get('id', ''),
                'candidate_name': cv.get('name', ''),
                'scores': scores
            })

        # Sort by overall score
        rankings.sort(key=lambda x: x['scores']['overall_score'], reverse=True)
        return rankings



## Test

In [ ]:
job_reqs = [
    '''Posisi: Kepala Koki (Executive Chef)
    #### Kualifikasi (Wajib):
    Pengalaman Kerja:
    Minimal 5 tahun bekerja di industri kuliner.
    Minimal 2 tahun memiliki pengalaman dalam peran kepemimpinan (misalnya Kepala Chef atau Sous Chef).
    IPK minimal 3.0.

    Keahlian Utama:
    Keahlian dalam Masakan:
    Harus memiliki keahlian dalam masakan Nusantara.
    Pengetahuan atau pengalaman dalam fusion dishes (hidangan fusi) akan menjadi nilai tambah.
    Keahlian Teknis:
    Terampil dalam pengembangan menu dan teknik memasak modern.
    Mampu mengelola anggaran dapur.
    Soft Skills:
    Memiliki kemampuan kuat dalam manajemen tim.

    Penghargaan dan Publikasi (Opsional, Diutamakan):
    Memiliki pencapaian yang diakui di bidang kuliner (misalnya, penghargaan, buku yang diterbitkan, atau artikel yang dipublikasikan).

    Pendidikan:
    Lulusan sekolah kuliner atau memiliki pengalaman kerja yang setara.'''
]


job_cv = [
    '''
---

**Informasi Pribadi**
- **Nama**: Paul Pogba
- **Alamat**: Jl. Cendana No 5
- **Nomor Telepon**: 0823121122112
- **Email**: emyu@gmail.com
- **Tempat, Tanggal Lahir**: Toulouse, 11 April 2002
- **Status Pernikahan**: Single

---

**Ringkasan Profesional**
Seorang Kepala Koki berpengalaman dengan lebih dari 7 tahun dalam dunia kuliner. Terampil dalam menciptakan menu kreatif, memimpin tim dapur, dan memastikan kualitas makanan terbaik. Berkomitmen terhadap kepuasan pelanggan, efisiensi operasional, dan inovasi dalam penyajian makanan.

---

**Pengalaman Kerja**

**Kepala Koki**
- **Nama Restoran atau Hotel**: Le Gourmet Paradise
- **Periode**: Januari 2018 – Desember 2023
  - Merancang dan mengembangkan menu yang menarik sesuai dengan tema restoran.
  - Mengawasi operasional dapur harian, termasuk pengadaan bahan baku, persiapan makanan, dan presentasi.
  - Melatih dan memotivasi tim dapur untuk menjaga standar kualitas dan efisiensi.
  - Mengelola anggaran dapur dan mengontrol biaya makanan untuk meningkatkan profitabilitas.
  - Menjaga kepatuhan terhadap standar keamanan pangan dan kebersihan.

**Sous Chef**
- **Nama Restoran atau Hotel**: The Culinary Haven
- **Periode**: Juli 2015 – Desember 2017
  - Mendukung Kepala Koki dalam pengelolaan operasional dapur.
  - Bertanggung jawab atas manajemen tim saat Kepala Koki tidak hadir.
  - Memastikan bahan baku tersedia dan sesuai dengan kebutuhan menu harian.
  - Membantu dalam inovasi resep dan presentasi makanan.

**Chef de Partie**
- **Nama Restoran atau Hotel**: Maison de Cuisine
- **Periode**: Maret 2012 – Juni 2015
  - Menyiapkan makanan sesuai pesanan dengan presisi dan kreativitas.
  - Mengawasi bagian dapur tertentu dan memastikan semua berjalan lancar.
  - Bekerjasama dengan tim untuk menjaga kualitas makanan yang konsisten.

---

**Pendidikan**
- **Diploma Seni Kuliner**: Akademi Kuliner Nusantara
  **Tahun Lulus**: 2011

- **Sertifikasi Keamanan dan Kebersihan Pangan**: Lembaga Sertifikasi Makanan Aman
  **Tahun Sertifikasi**: 2013

---

**Keahlian**
- Pengembangan Menu dan Resep
- Manajemen Operasional Dapur
- Kepemimpinan dan Pelatihan Tim
- Pengendalian Biaya dan Inventaris
- Presentasi dan Plating Makanan
- Pengetahuan tentang Berbagai Masakan Internasional dan Tradisional

---

**Bahasa**
- **Bahasa Indonesia**: Fasih
- **Bahasa Inggris**: Profesional

---

**Referensi**
Referensi tersedia berdasarkan permintaan.

---

Demikian CV ini saya buat dengan sebenar-benarnya dan dapat dipertanggungjawabkan.

Paul Pogba
(Signature jika diperlukan)

---''',
    '''
    Curriculum Vitae
    Nama: Aditya Pratama
    Alamat: Jalan Melati No. 12, Jakarta, Indonesia
    Telepon: +62 812 3456 7890
    Email: adityapratama@gmail.com

    Profil
    Saya seorang Electrical Engineer yang berdedikasi dengan pengalaman lebih dari 5 tahun dalam desain, pengembangan, dan pemeliharaan sistem kelistrikan. Memiliki keahlian dalam analisis sistem tenaga, desain sirkuit, dan manajemen proyek. Saya bersemangat untuk memberikan solusi inovatif dan efisien untuk tantangan teknis.

    Pendidikan
    Sarjana Teknik Elektro
    Universitas Indonesia, Depok
    2013 - 2017

    IPK: 3.80
    IPK 3.80
    Fokus: Sistem Tenaga dan Elektronika
    Pengalaman Kerja
    Electrical Engineer
    PT Energi Nusantara, Jakarta
    2018 - Sekarang

    Mendesain dan mengawasi instalasi sistem kelistrikan untuk proyek skala besar seperti gedung perkantoran dan fasilitas industri.
    Meningkatkan efisiensi sistem dengan menganalisis dan mengoptimalkan desain sirkuit menggunakan perangkat lunak simulasi.
    Memimpin tim teknisi dalam pengujian dan pemeliharaan sistem kelistrikan.
    Junior Electrical Engineer
    PT Teknologi Listrik, Bandung
    2017 - 2018

    Membantu dalam perancangan diagram kelistrikan untuk sistem distribusi tenaga.
    Melakukan inspeksi rutin untuk memastikan keamanan dan efisiensi instalasi listrik.
    Menyusun laporan teknis dan memberikan rekomendasi perbaikan.
    Keahlian
    Analisis Sistem Tenaga
    Desain Sirkuit dan PCB
    Pemrograman PLC dan HMI
    Penggunaan perangkat lunak teknik: AutoCAD, MATLAB, ETAP
    Pengetahuan mendalam tentang standar kelistrikan nasional dan internasional (SNI, IEC).
    Sertifikasi
    Certified Energy Manager (CEM), 2021
    Sertifikat Kompetensi Ahli K3 Listrik, Kementerian Ketenagakerjaan RI, 2019
    Proyek Terkini
    Peningkatan Efisiensi Energi di Pabrik Tekstil: Mengurangi konsumsi energi hingga 20% melalui optimasi sistem kelistrikan.
    Desain Sistem Kelistrikan Gedung Perkantoran 25 Lantai: Merancang sistem distribusi tenaga yang hemat energi dan sesuai dengan standar keselamatan.
    Bahasa
    Bahasa Indonesia: Fasih
    Bahasa Inggris: Mahir
    '''
]



In [ ]:
import warnings
warnings.filterwarnings("ignore")

matcher = IndonesianCVMatcherWithNormalizer(job_req_dict={})

# Load your CVs and job requirement (in Indonesian)
cvs = [
    {'id': '1', 'name': 'Budi', 'text': job_cv[1]},
    # ... more CVs
]
job_req = job_reqs[0]

# Get rankings
similarity = matcher.calculate_similarity_scores(cvs[0]['text'], job_req, mode='embedding')
print(similarity)
# rankings = matcher.rank_candidates(cvs, job_req)

matcher.print_match_requirements(cvs[0]['text'], job_reqs[0])

In [ ]:
import os

In [ ]:
cvs = [
    {'id': '1', 'name': 'Budi', 'text': job_cv[0]},
    # ... more CVs
]
job_req = job_reqs[0]

# Get rankings
similarity = matcher.calculate_similarity_scores(cvs[0]['text'], job_req, mode='embedding')
print(similarity)
# rankings = matcher.rank_candidates(cvs, job_req)

matcher.print_match_requirements(cvs[0]['text'], job_reqs[0])

In [ ]:
folder_path = "/content/drive/MyDrive/Data/A data collection"

cv_texts = []

for file in os.listdir(folder_path):
    if file.endswith(".txt"):
        with open(os.path.join(folder_path, file), "r") as f:
            cv_text = f.read()
            cv_texts.append(cv_text)


for i in range(5):
    print(cv_texts[i])
    print("\n<===========================>\n")

## Test with 200 datasets

In [ ]:
output_path = "200_data_test_results.txt"

with open(output_path, "w") as f:
    for i, cv in enumerate(cv_texts):
        f.write(f"## Data CV number {i+1}\n")

        similarity = matcher.calculate_similarity_scores(cv, job_req, mode='embedding')
        print(similarity)
        f.write(f"{similarity}\n")

        f.write("--------------------------------------------------------------------\n")

        # Modify `print_match_requirements` to write directly to the file
        matcher.print_match_requirements(cv, job_req, file_handle=f)

        f.write("\n<=====================================================================>\n")


Execution time: 1.0950 seconds
{'semantic_similarity': 0.8801729083061218, 'skills_similarity': 0.746669352054596, 'keyword_overlap': 0.1625, 'entity_match_score': 0.0, 'raw_overall_score': 0.5991771918535232, 'normalized_overall_score': 1.0}
Execution time: 1.0659 seconds
{'semantic_similarity': 0.8605464696884155, 'skills_similarity': 0.732119619846344, 'keyword_overlap': 0.1125, 'entity_match_score': 0.0, 'raw_overall_score': 0.5780110573768614, 'normalized_overall_score': 0.7456131238192933}
Execution time: 0.7710 seconds
{'semantic_similarity': 0.8860381841659546, 'skills_similarity': 0.7619936466217041, 'keyword_overlap': 0.1375, 'entity_match_score': 0.0, 'raw_overall_score': 0.6051974618434904, 'normalized_overall_score': 1.0}
Execution time: 0.7617 seconds
{'semantic_similarity': 0.8717107176780701, 'skills_similarity': 0.6449302434921265, 'keyword_overlap': 0.15, 'entity_match_score': 0.0, 'raw_overall_score': 0.525036706328392, 'normalized_overall_score': 0.10158636731663244

In [ ]:
matcher.export()